# A Simple Example in Parallel Computing with Python
## -- an example in weighted average calculating
by Xiaodong (Sheldon) Zhai,  xz125@duke.edu

### Randomly generate some data given size of file

In [197]:
import os, time
import numpy as np

# number of items in a data matrix
n = int(2e6)

# generate prices and volume data, and save the whole matrix in to a .npy file
if os.path.isfile('paralle_million_data_{0}.npy'.format(n)):
    print('data file exists')
    mat = np.load('paralle_million_data_{0}.npy'.format(n))
    if len(mat) != n:
        prices = np.random.randint(201000, 2010500, size=(n, 1))
        volume = np.random.randint(100, 200, size=prices.shape)
        mat = np.array(list(zip(prices[:,0], volume[:,0])))
        print('new data file generated, with size ', n)
        np.save('paralle_million_data_{0}.npy'.format(n), mat)      
else:
    print('data file does not exist, new file generated')
    prices = np.random.randint(201000, 2010500, size=(n, 1))
    volume = np.random.randint(100, 200, size=prices.shape)
    mat = np.array(list(zip(prices[:,0], volume[:,0])))
    np.save('paralle_million_data_{0}.npy'.format(n), mat)
mat.shape

data file exists


(2000000, 2)

### import library 

In [198]:
import multiprocessing 
from multiprocessing import cpu_count, Pool, Process, Queue
cpu_count()

8

### define the naive vwap calcualting function

In [199]:
def vwap(mat, window=5):
    vwap = np.zeros(mat.shape[0])
    for i in range(window, len(mat)):
        weight = mat[i-window:i+1, 1]
        vwap[i] = np.sum(mat[i-window:i+1, 0] * weight)/weight.sum()
    return vwap

### banchamrk time, without multiprocess optimization

In [200]:
%%time
# banchmark time, without multiprocess optimization
vwap_0 = vwap(mat, window=5)

CPU times: user 16.3 s, sys: 0 ns, total: 16.3 s
Wall time: 16.3 s


### time with Pool  

In [201]:
%%time
pool = Pool(processes=cpu_count())
res = [pool.apply_async(vwap, (mat, 5,))]
vwap_1 = np.array([r.get() for r in res])
pool.close()
pool.join()

CPU times: user 30.9 ms, sys: 58.6 ms, total: 89.6 ms
Wall time: 14 s


### it does not work significantly!
According to numerous tests and materials from the Internet, 
this is highly probable due to the Global Interpreter Lock (GIL) which may mean that only one line of code can 
be compiled in Python interpretor -- thus we cannot improve the performance.

### try Process to avoid GIL

#### Some clarification
Here, because the calculation of vwap requires a 'communicaiton' between processes, we need to modify our
functions of vwap, and a new mp_vwap() to be defined to manage the processes.

###### the zeros beginning of vwap
Firstly, we observe that given a vwap(mat, window), the returned vwap vector starts with several zeros this 
is because for the first window vwaps we do not have enough sample to conduct the calculation.

###### communicaiton between processes
The vwap calculating is not _independent_ due to the vwap[t] requires its neighbours of volume window, thus
simply splitting the matrix/order_book into parts would lead to the fact that
each process does not have the first several prices and volumes data to calculate, 
whic makes _zeros beginning_ for each part.

###### how to solve this
With those being said, we just need to let each process to handle data with extra previous window rows,
so that the zeros beginning will be eliminated.
And the modifited vwap function can return a chunkated vwap vector with zeros ignored.
Also, for the very first process to handle the beginning part of data, we can conncatenate it with some 
zeros with respect to right corresponding window.




In [202]:
def mp_vwap(mat, nprocs, window=3):
    
    
    # define a new vwap funciton, the only difference is the returned part
    # of course, the logic behind has also changed: to handle mat with extra headers
    def vwap2(mat, win, ):
        vwap = np.zeros(len(mat))
        for i in range(win, len(mat)):
            weight = mat[i-win:i+1, 1]
            vwap[i] = np.sum(mat[i-win:i+1, 0] * weight)/weight.sum()
        return vwap[win:] 
        
    def worker2(mat, uid, out_q, win):
        outdict = {}
        if uid == 0:
            # if this is the first process, we add the chunkated zeros back
            outdict[uid] = np.hstack([[0]*win, vwap2(mat, win)])
        else:
            outdict[uid] = vwap2(mat, win)
        out_q.put(outdict)
    
    
    
    out_q = Queue()
    chunksize = int(np.ceil(len(mat) / float(nprocs)))
    procs = []

    for i in range(nprocs):
        p = Process(
            target=worker2,
            args=(mat[np.maximum(chunksize * i - window, 0) : chunksize * (i + 1)],
                  i,
                  out_q,
                  window))
        procs.append(p)
        p.start()

    
    resultdict = {}
    for i in range(nprocs):
        resultdict.update(out_q.get())

    # Wait for all worker processes to finish
    for p in procs:
        p.join()

    return resultdict



In [205]:
%%time
tmp = mp_vwap(mat, nprocs=8, window=5)
# collect results from processes
vwap_2 = []
for k,v in tmp.items():
    vwap_2.extend(v)
vwap_2 = np.array(vwap_2)

CPU times: user 299 ms, sys: 77.6 ms, total: 377 ms
Wall time: 2.44 s


#### if two vwaps are the same

In [206]:
np.all(vwap_0 == vwap_2)

True

#### So, nice job!